# Entrenamiento Modelo - ResNet18 + Metadatos 
Este notebook entrena un modelo **multiclase** con imágenes 224×224 + metadatos.

In [ ]:

# Comprobar GPU y entorno
import torch, sys
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


In [ ]:

# Instalar dependencias
!pip -q install mlflow charset-normalizer pandas scikit-learn matplotlib

In [ ]:

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')
PROJECT_DIR = "/content/drive/MyDrive/ProyectoFinalKC"
import os, pathlib
pathlib.Path(PROJECT_DIR).mkdir(parents=True, exist_ok=True)
os.chdir(PROJECT_DIR)
print("Working dir:", os.getcwd())


In [ ]:

# Estructura de carpetas esperada
import os, pathlib
pathlib.Path("General").mkdir(parents=True, exist_ok=True)
pathlib.Path("Data/dataset").mkdir(parents=True, exist_ok=True)
print("Estructura creada/chequeada.")


In [ ]:

# Imports y configuración
import sys
from pathlib import Path
from datetime import datetime
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T
import mlflow, tempfile, shutil
from torch.utils.data import Subset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

sys.path.append(str(Path("General").resolve()))
from load_dataloaders_Leti import load_dataloaders

BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "Data" / "dataset"
TRAIN_PT = str(DATA_DIR / "train_dataset.pt")
VAL_PT   = str(DATA_DIR / "val_dataset.pt")
TEST_PT  = str(DATA_DIR / "test_dataset.pt")

BATCH_SIZE   = 64
EPOCHS       = 20
LR           = 1e-4
WEIGHT_DECAY = 5e-4
NUM_WORKERS  = 2
SEED         = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pin_mem = device.type == "cuda"
print("Device:", device)


In [ ]:
def set_seed(seed: int = 42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True


In [ ]:
# Utilities

def get_train_transform(img_size: int = 224):
    return T.Compose([
        T.RandomHorizontalFlip(p=0.5),
        T.RandomRotation(10),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406],
                    std =[0.229, 0.224, 0.225]),
        T.RandomErasing(p=0.1)
    ])

set_seed(SEED)

# Cargar DataLoaders desde tus Subsets .pt
train_loader, val_loader, test_loader = load_dataloaders(
    TRAIN_PT, VAL_PT, TEST_PT,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=pin_mem,
    seed=SEED
)

# Sobrescribir transform SOLO para train
base_train_ds = train_loader.dataset.dataset  
base_train_ds.transform = get_train_transform(224)
print("Transform de train aplicada:", base_train_ds.transform)

# Inferir meta_dim, num_classes y class weights
train_subset = train_loader.dataset
sample = train_subset[0]
meta_dim = int(sample[1].shape[0])

def get_train_labels(train_subset):
    base = train_subset.dataset
    idxs = train_subset.indices
    import numpy as np
    return np.array([int(base.targets[i]) for i in idxs], dtype=int)

y_train = get_train_labels(train_subset)

num_classes = int(y_train.max() + 1)
class_names = [str(i) for i in range(num_classes)]

def compute_class_weights(y, num_classes):
    import numpy as np, torch
    counts = np.bincount(y, minlength=num_classes).astype(np.float64)
    w = 1.0 / (counts + 1e-6)
    w = w * (num_classes / w.sum())
    return torch.tensor(w, dtype=torch.float32)

class_weights = compute_class_weights(y_train, num_classes).to(device)

print(f"num_classes={num_classes}, meta_dim={meta_dim}, train_size={len(train_subset)}")

In [ ]:

# Modelo (ResNet18 + MLP metadatos)
class ImageClassifier(nn.Module):
    def __init__(self, meta_dim: int, num_classes: int, pretrained: bool = True, dropout: float = 0.6):
        super().__init__()
        weights = ResNet18_Weights.IMAGENET1K_V1 if pretrained else None
        self.backbone = resnet18(weights=weights)
        in_feats = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        self.meta = nn.Sequential(
            nn.Linear(meta_dim, 32),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(32),
        )

        self.head = nn.Sequential(
            nn.Linear(in_feats + 32, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )

    def forward(self, x_img, x_meta):
        f_img = self.backbone(x_img)
        f_meta = self.meta(x_meta)
        f = torch.cat([f_img, f_meta], dim=1)
        return self.head(f)


In [ ]:
# Modelo + criterio + optimizador
model = ImageClassifier(meta_dim=meta_dim, num_classes=num_classes, pretrained=True, dropout=0.6).to(device)

# Regularización: class weights + label smoothing
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.10)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))

In [ ]:
# Funciones de train/eval con métricas (acc, F1, precision, recall)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from pathlib import Path

def plot_confusion(cm, class_names, out_path: Path):
    fig = plt.figure()
    plt.imshow(cm, interpolation="nearest")
    plt.title("Matriz de confusión"); plt.colorbar()
    ticks = range(len(class_names))
    plt.xticks(ticks, class_names, rotation=45, ha="right")
    plt.yticks(ticks, class_names)
    plt.tight_layout(); plt.ylabel("Real"); plt.xlabel("Predicción")
    fig.savefig(out_path, bbox_inches="tight"); plt.close(fig)

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    ys_all, ps_all = [], []
    for imgs, metas, ys in loader:
        imgs = imgs.to(device, non_blocking=True)
        metas = metas.to(device, non_blocking=True)
        ys   = ys.to(device, non_blocking=True, dtype=torch.long)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            logits = model(imgs, metas)
            loss   = criterion(logits, ys)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(1)
        ys_all.append(ys.detach().cpu()); ps_all.append(preds.detach().cpu())

    loss_ep = total_loss / len(loader.dataset)
    y = torch.cat(ys_all).numpy(); p = torch.cat(ps_all).numpy()
    acc = accuracy_score(y, p); f1m = f1_score(y, p, average="macro")
    return loss_ep, acc, f1m

@torch.no_grad()
def evaluate(model, loader, criterion, device, prefix, class_names, out_dir: Path):
    model.eval()
    out_dir.mkdir(parents=True, exist_ok=True)

    total_loss = 0.0
    ys_all, ps_all = [], []
    for imgs, metas, ys in loader:
        imgs = imgs.to(device, non_blocking=True)
        metas = metas.to(device, non_blocking=True)
        ys   = ys.to(device, non_blocking=True, dtype=torch.long)
        logits = model(imgs, metas)
        loss = criterion(logits, ys)
        total_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(1)
        ys_all.append(ys.detach().cpu()); ps_all.append(preds.detach().cpu())

    loss_ep = total_loss / len(loader.dataset)
    y = torch.cat(ys_all).numpy(); p = torch.cat(ps_all).numpy()
    acc   = accuracy_score(y, p)
    f1m   = f1_score(y, p, average="macro")
    precm = precision_score(y, p, average="macro", zero_division=0)
    recm  = recall_score(y, p, average="macro", zero_division=0)

    # Artefactos (report + cm)
    rep_path = out_dir / f"{prefix}_classification_report.txt"
    with open(rep_path, "w", encoding="utf-8") as f:
        f.write(classification_report(y, p, digits=3))

    cm = confusion_matrix(y, p)
    cm_path = out_dir / f"{prefix}_confusion_matrix.png"
    plot_confusion(cm, class_names, cm_path)

    return {
        "loss": loss_ep, "acc": acc, "f1_macro": f1m,
        "precision_macro": precm, "recall_macro": recm,
        "rep_path": str(rep_path), "cm_path": str(cm_path)
    }

In [ ]:
# MLflow: setup y logging
mlruns_dir = Path.cwd() / "mlruns"             
mlflow.set_tracking_uri("file:" + str(mlruns_dir.resolve()))
mlflow.set_experiment("Experimento_Leticia_ResNet18_Colab")

RUN_NAME = "colab_resnet18_mm_" + datetime.now().strftime("%Y%m%d_%H%M%S")

mlflow.start_run(run_name=RUN_NAME)
mlflow.log_params({
    "batch_size": BATCH_SIZE, "epochs": EPOCHS,
    "lr": LR, "weight_decay": WEIGHT_DECAY,
    "num_workers": NUM_WORKERS, "pin_memory": pin_mem,
    "label_smoothing": 0.10,
    "meta_dim": meta_dim, "num_classes": num_classes,
    "backbone": "resnet18_imagenet", "dropout": 0.6,
})

# Carpeta temporal para artefactos (se suben a MLflow y se borran)
tmp_art_dir = Path(tempfile.mkdtemp(prefix="mlflow_artifacts_"))
tmp_art_dir

In [ ]:
# Train loop con early stopping y logging a MLflow
best_val_f1 = -1.0
patience = 6
bad = 0

ckpt_path = tmp_art_dir / "best_model.pt"  # se subirá a MLflow al mejorar

for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc, tr_f1m = train_one_epoch(model, train_loader, criterion, optimizer, device)
    va_out = evaluate(model, val_loader, criterion, device, prefix="val",
                      class_names=class_names, out_dir=tmp_art_dir / "reports_val")

    print(f"[Época {epoch:02d}/{EPOCHS}] "
          f"train_loss={tr_loss:.4f} acc={tr_acc:.4f} f1m={tr_f1m:.4f} | "
          f"val_loss={va_out['loss']:.4f} acc={va_out['acc']:.4f} f1m={va_out['f1_macro']:.4f}")

    mlflow.log_metrics({
        "train_loss": tr_loss, "train_acc": tr_acc, "train_f1m": tr_f1m,
        "val_loss": va_out["loss"], "val_acc": va_out["acc"], "val_f1m": va_out["f1_macro"],
        "val_prec_m": va_out["precision_macro"], "val_rec_m": va_out["recall_macro"],
    }, step=epoch)

    # Guardar mejor
    if va_out["f1_macro"] > best_val_f1 + 1e-6:
        best_val_f1 = va_out["f1_macro"]; bad = 0
        torch.save(model.state_dict(), ckpt_path)
        # Subir a MLflow artefactos de val
        mlflow.log_artifact(str(ckpt_path))
        mlflow.log_artifact(va_out["rep_path"])
        mlflow.log_artifact(va_out["cm_path"])
    else:
        bad += 1
        if bad >= patience:
            print("Early stopping!")
            break

In [ ]:
# Test final + cierre de MLflow
# Cargar mejor modelo si existe
if ckpt_path.exists():
    model.load_state_dict(torch.load(ckpt_path, map_location="cpu"))
    model.to(device)

# En test quitamos smoothing pero mantenemos class weights
criterion_eval = nn.CrossEntropyLoss(weight=class_weights)

test_out = evaluate(model, test_loader, criterion_eval, device, prefix="test",
                    class_names=class_names, out_dir=tmp_art_dir / "reports_test")

print(f"[TEST] loss={test_out['loss']:.4f} acc={test_out['acc']:.4f} f1m={test_out['f1_macro']:.4f}")

# Log a MLflow
mlflow.log_metrics({
    "test_loss":   test_out["loss"],
    "test_acc":    test_out["acc"],
    "test_f1m":    test_out["f1_macro"],
    "test_prec_m": test_out["precision_macro"],
    "test_rec_m":  test_out["recall_macro"],
})

mlflow.log_artifact(test_out["rep_path"])
mlflow.log_artifact(test_out["cm_path"])

# Limpieza local y cierre del run
shutil.rmtree(tmp_art_dir, ignore_errors=True)
mlflow.end_run()

print("Artefactos y métricas guardados en:", mlruns_dir)